### Keeping your API keys secret

Often, to access data from APIs, you must use an API "key". This key identifies your account to the API you're calling. 

You **never** want to commit your API keys for any service to `git`, especially if your repo is public. Anyone reading your code could take that API key and make API requests on your behalf. If your key is tied to a service like Amazon Web Services, where you can create resources (*that cost money*) using the API, making your key public can be a costly mistake.

Since you want to commit your _code_ to `git`, this means you need a way to separate your code from your API key.

Typically, you'll want to keep your API key in some file on your computer (for some services, it may make sense to encrypt your API key, which is beyond the scope of this document). Below, we'll run through an example of how to pull an API key from a file into a Python program, as a string that you can use in your program.

Before you run the code below, 

* Create a file called `apikey`, with a single line that contains the API key you want to store
* Create a `.gitignore` file, add a single line with the word "apikey" and save the file. `.gitignore` is a special file that `git` uses to ignore any files listed. You won't be asked to add / commit changes to these files, so it's an easy way to make sure you don't accidentally commit these files to `git`. See [this reference](https://git-scm.com/docs/gitignore) for more information.

In [ ]:
# See above - we've created an API key in a file called 'apikey', 
# in the same directory as this notebook
filename = 'apikey'

In [ ]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
api_key = get_file_contents(filename)
print("Our API key is: %s" % (api_key))